In [43]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models import resnet18, vgg16
import torch.nn.functional as functional

In [17]:
#import resnet18 (pretrained)
ResNet18 = torchvision.models.resnet18(pretrained=True)
#retrieve all layers but the last two
modules=list(ResNet18.children())#[:-2]
#modules=list(resnet18.children())
#redefine the network to have all layers but the last two
ResNet18=nn.Sequential(*modules) #*list: unpack

In [18]:
print(ResNet18)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [48]:
class ResNet18(torch.nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        #leave out final two layers (avgsize-pooling and flattening)
        features = list(resnet18(pretrained = True).children())[:-2]
        self.features = nn.ModuleList(features).eval() 
        
    def forward(self, x):
        results = []
        #go through modules
        for ii,model in enumerate(self.features):
            #forward propagation
            x = model(x)
            #take 4th module's activations
            if ii in {4}:
                results.append(x)
        #upsample (rescale to same size)
        x = functional.interpolate(x, size=(100,100), mode="bilinear")
        for i in range(len(results)):
            #upsample (rescale to same size)
            intermediate = functional.interpolate(results[i], size=(100,100), mode="bilinear")
            #append to other output along feature channel dimension
            x = torch.cat((x,intermediate), 1)
        #return combined activations
        return x
    
net = ResNet18()

In [49]:
#get intermediate and final output
output = net(torch.randn(1,3,400,400))

#upsample (rescale to same size)
#intermediate = functional.interpolate(intermediate[0], size=(100,100), mode="bilinear")
#output = functional.interpolate(output, size=(100,100), mode="bilinear")

#combine together in dimension with index 1 (feature channels)
#total = torch.cat((output, intermediate), 1)
print(output.size())
#print(intermediate.size())
#print(total.size())

0
1
2
3
4
5
6
7
torch.Size([1, 576, 100, 100])


In [24]:
list(resnet18(pretrained = True).children())[4][1].conv1

Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [55]:
VGG16 = vgg16(pretrained=True)
modules = list(VGG16.features)
VGG16 = nn.Sequential(*modules)
print(VGG16)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [61]:
class MyVGG19(torch.nn.Module):
    def __init__(self):
        super(MyVGG19, self).__init__()
        #leave out final two layers (avgsize-pooling and flattening)
        features = list(torchvision.models.vgg19(pretrained = True).features)
        self.features = nn.ModuleList(features).eval() 
        
    def forward(self, x):
        results = []
        #go through modules
        for ii,model in enumerate(self.features):
            #forward propagation
            x = model(x)
            #take 4th module's activations
            if ii in {2,4,6}:
                results.append(x)
        #upsample (rescale to same size)
        x = functional.interpolate(x, size=(100,100), mode="bilinear")
        #x = functional.interpolate(results[0], size=(100,100), mode="bilinear")
        for i in range(len(results)):
            #upsample (rescale to same size)
            intermediate = functional.interpolate(results[i], size=(100,100), mode="bilinear")
            #append to other output along feature channel dimension
            x = torch.cat((x,intermediate), 1)
        #return combined activations
        return x

In [62]:
model = MyVGG19()
print(model)

MyVGG19(
  (features): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inpl